# Workshop on PDMP

In this worshop we will 

- Example Random walk vs lifted random walk. 
- Reproduce the plot 
- Plot the autocorrelation function. 

- Simulate data from the linear regression (1 dimensional case  case)
- Sample from a quadratic potential 
- Plot traces, plot empirical distribution (histogram) 
### Random Walk
Simulate and save the trajectory of a random walk on a lattice $\{1,2,\dots,N\}$. The random walk move from $x 
\to x \pm 1$ with probability $1/2$. 

In [1]:
# Required libraries
import math
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# template for random walk 

In [27]:
# SOLUTIONS WITH MARKS
# CHECK AND ADAPT AND ADD BOUNDARIES
def random_walk(steps, initial_state, boundary):
    state = initial_state
    trajectory = [state]
    for _ in range(steps):
        step = np.random.choice([-1, 1])
        if abs(state + step) < boundary:
            state += step
        trajectory.append(state)
    return trajectory

# Simulate random walk with 1000 steps
steps = 1000
positions = random_walk(steps, initial_state, boundary):


TypeError: random_walk() missing 2 required positional arguments: 'initial_state' and 'boundary'

### Lifted Random Walk
Simulate and save....

In [ ]:
# Template lifted random walk


In [ ]:
# SOLUTIONS WITH MARKS 
def lifted_random_walk(steps, initial_state, initial_velocity, boundary, refresh_prob):
    state = initial_state
    velocity = initial_velocity
    trajectory = [state]
    for _ in range(steps):
        if (refresh_prob > np.random.random()) and (abs(state) < boundary):
            velocity *= -1
        elif state * velocity == boundary :     
            velocity *= -1
        else: 
            state += velocity
        trajectory.append(state)
    return trajectory

traj = lifted_random_walk(50, 0, 1, 5, 0.1)
print(traj)

[0, 1, 2, 3, 3, 2, 1, 0, -1, -2, -3, -4, -5, -5, -4, -3, -3, -4, -4, -3, -2, -2, -3, -4, -5, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 5, 4, 3, 2, 1, 0, -1, -2, -3, -4, -5, -5, -4, -3, -2]


# Plotting autocorrelation and traces
- reproduce the trace plot 
- Compute and compare autocorrelation of rw and lrw


In [ ]:
# template for plots 
# use numpy.correlate

In [ ]:
# SOLUTIONS WITH MARKS 

Simulate data from linear regression with one covariate
- simulate $N=100$ observations:
- $x_i \sim \mathcal{N}(0,1)$
- $y_i \sim \mathcal{N}(x_i\beta, \sigma^2)$, $\sigma = 1.0, \beta = 0.5$

In [ ]:
# template for simulating data 
# use numpy.random.normal
# store y = (y_1,y_2,...,y_N) as a numpy.array 
#  ...


In [ ]:
# SOLUTIONS WITH MARKS
# CHECK AND ADAPT
import numpy as np

def simulate_linear_regression_data(N, true_slope, true_intercept, noise_std):
    # Generate random covariate values
    X = np.random.rand(N)
    
    # Generate response variable values
    noise = np.random.normal(0, noise_std, N)
    y = true_slope * X + true_intercept + noise
    
    return X, y

# Parameters
N = 100  # Number of data points
true_slope = 2.0  # True slope parameter
true_intercept = 1.0  # True intercept parameter
noise_std = 0.5  # Standard deviation of noise

# Simulate data
X, y = simulate_linear_regression_data(N, true_slope, true_intercept, noise_std)

# Display the first few data points
print("Covariate (X):", X[:5])
print("Response (y):", y[:5])

Covariate (X): [0.92431025 0.41821823 0.15416846 0.91175336 0.58510597]
Response (y): [2.47278991 1.89052296 2.12070938 2.86163345 2.34201815]


# Sample from the quadratic potential 
- In this case $\lambda(t) = max(0, a + b*t)$
- simulate $\tau$ such that $P(\tau > t) = \exp(- \int_0^t \lambda(s) ds)$


In [ ]:
# provide the function 
# \lambda(t) = max(0, a + b*t)
#  simulate \tau such that P(\tau > t) = \exp(- \int_0^t \lambda(s) ds)
def poisson_time(a, b, u):
    if b > 0:
        if a < 0:
            return math.sqrt(-math.log(u)*2.0/b) - a/b
        else:  # a > 0
            return math.sqrt((a/b)**2 - math.log(u)*2.0/b) - a/b
    elif b == 0:
        if a > 0:
            return -math.log(u)/a
        else:
            return float('inf')
    else:  # b < 0
        if a <= 0:
            return float('inf')
        elif -math.log(u) <= -a**2/b + a**2/(2*b):
            return -math.sqrt((a/b)**2 - math.log(u)*2.0/b) - a/b
        else:
            return float('inf')

# Example usage
result = poisson_time(2, 3, 0.5)
print(result)

0.28545862173698733


# Alternative simulation of PDMP bounce/event times for a 1D Gaussian

A variant on the following might be useful as it allows us to simulate the bounce coordinates in one line of code without the students having to worry too much about the velocity, though we **do need to add something to track its sign**...

- At constant unit speed, the distance travelled $|\eta|$ (before the next event) through the uphill part of some 1D potential $U$ can be simulated via:
    $$
    \rm{rand}(0, 1] = \exp \left[- \int_0^{\eta} \left(\frac{\partial U}{\partial x}\right)^+ dx \right]
    $$
- As we're simulating the distance travelled through the uphill part of the potential, we can invert this to write:
    $$
    -\ln \left( \rm{rand}(0, 1] \right) = U(\eta) - U(0)
    $$
- For a 1D mean-zero, unit-variance Gaussian $\mathcal{N}(0, 1)$ this becomes:
    $$
    |\eta| = \sqrt{- 2 \ln \left( \rm{rand}(0, 1] \right)}
    $$
- More generally, for $\mathcal{N}(\mu, \sigma^2)$:
    $$
    -\ln \left( \rm{rand}(0, 1] \right) = \frac{1}{2\sigma^2} (\eta^2 - 2 \mu \eta)
    $$
    $$
    \Rightarrow |\eta| = \mu + \sqrt{\mu^2 - 2 \sigma^2 \ln \left( \rm{rand}(0, 1] \right)}
    $$

In [ ]:
def get_distance_from_origin_to_next_event_for_1d_gaussian(mean, variance):
        r"""
        Returns the distance $|\eta|$ travelled (before the next zig-zag event) through the uphill part of
        a one-dimensional Gaussian $\mathcal{N}(mu, sigma^2)$ with mean $\mu$ and variance $\sigma^2$, 
        i.e., from the origin to $\eta$.  This is calculated by inverting

            $ \rand(0.0, 1.0) =
                \exp \left[- \int_0^{\eta} \left(\frac{\partial U}{\partial x}\right)^+ dx \right] $
        
        Parameters
        ----------
        mean : float
            The mean ($\mu$) of the Gaussian target distribution.
        variance : float
            The variance ($\sigma^2$) of the Gaussian target distribution.

        Returns
        -------
        float
            The distance travelled (before the next zig-zag event) through the uphill part of the 
            one-dimensional Gaussian potential $\mathcal{N}(mean, variance)$.
        """
        return mean + (mean ** 2.0 - 2.0 * variance * np.log(1.0 - np.random.random())) ** 0.5

- Original code for a 1D mean-zero, unit-variance Gaussian $\mathcal{N}(0, 1)$ (will remove later):

In [ ]:
def get_distance_from_origin_to_next_event_for_1d_gaussiant_mean_zero_variance_one():
        r"""
        Returns the distance $|\eta|$ travelled (before the next zig-zag event) through the uphill part of
        a one-dimensional Gaussian with mean 0, variance 1, i.e., from the origin to $\eta$.  This is 
        calculated by inverting

            $ \rand(0.0, 1.0) =
                \exp \left[- \int_0^{\eta} \left(\frac{\partial U}{\partial x}\right)^+ dx \right] $

        Returns
        -------
        float
            The distance travelled (before the next zig-zag event) through the uphill part of the 
            one-dimensional Gaussian potential N(0, 1).
        """
        return (- 2.0 * np.log(1.0 - np.random.random())) ** 0.5


### Derive analyitically posterior 
- Set a Gaussian proir
- Derive the Gaussian posterior 
- What is its mean?
image.png


### Simulate PDMP and plot trace and compute empirical mean
- Fill in the gaps in the tamplate
- Figure how to plot the trace
- Figure how to derive the mean from the skeleton points 

![image](procedure_zz.png)